In [42]:
import cv2
import imutils
import numpy as np
import pytesseract

In [43]:
img = cv2.imread('/home/mark/programs/python_programs/license_plate_recognition/license_plates/20220125_124416.jpg',
                 cv2.IMREAD_COLOR)  # Path to image
img = cv2.resize(img, (600, 400))

In [44]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 13, 15, 15)

In [45]:
edged = cv2.Canny(gray, 30, 200)
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
screenCnt = None

In [46]:
for c in contours:

    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)

    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print("No contour detected")
else:
    detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)


In [47]:
mask = np.zeros(gray.shape, np.uint8)
new_image = cv2.drawContours(mask, [screenCnt], 0, 255, -1, )
new_image = cv2.bitwise_and(img, img, mask=mask)
#new_image

In [52]:
(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx + 1, topy:bottomy + 1]

In [53]:
config = "--psm 7 -c tessedit_char_whitelist=0123456789KMT"
text = pytesseract.image_to_string(Cropped, config=config) 
#print("programming_fever's License Plate Recognition\n")
#print("Detected license plate Number is:", text)

text

'\x0c'